# Fine-tuning paso a paso: Llama-3.2-1B con AmericasNLP y diccionario estructurado

Este notebook guía el proceso completo y reproducible para ajustar un modelo LLM (meta-llama/Llama-3.2-1B) usando HuggingFace Transformers, con datos de AmericasNLP y un diccionario estructurado.

## 1. Importar librerías necesarias

Instala y carga todas las librerías requeridas para el fine-tuning reproducible.

In [4]:
%pip uninstall -y transformers accelerate peft sentence-transformers

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: -y


In [1]:
%pip uninstall -y torch torchvision torchaudio
%pip install torch==2.2.2+cu121 torchvision==0.17.2+cu121 torchaudio==2.2.2+cu121 --index-url https://download.pytorch.org/whl/cu121

# Verificar instalación y soporte CUDA
import torch
print('torch version:', torch.__version__)
print('CUDA disponible:', torch.cuda.is_available())
print('Versión CUDA (PyTorch):', torch.version.cuda)
print('Dispositivos CUDA:', torch.cuda.device_count())
if torch.cuda.is_available():
    print('Nombre GPU:', torch.cuda.get_device_name(0))

Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 2.5/2.5 GB 580.4 kB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 32.8 MB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 28.8 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 32.6 MB/s eta 0:00:00
     ------------------------------------- 199.3/199.3 kB 11.8 MB/s eta 0:00:00
     -------------------------------------- 134.9/134.9 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 31.0 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 29.7 MB/s eta 0:00:00
     --------------------------------------- 12.9/12.9 MB 29.8 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 21.7 MB/s eta 0:00:00
     ------------------------------------- 536.2/536.2 kB 17.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\walte\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\walte\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <mo

torch version: 2.2.2+cu121
CUDA disponible: True
Versión CUDA (PyTorch): 12.1
Dispositivos CUDA: 1
Nombre GPU: NVIDIA GeForce GTX 1070


In [1]:
%pip install -U transformers datasets accelerate


[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install transformers accelerate peft sentence-transformers

     --------------------------------------- 12.0/12.0 MB 27.3 MB/s eta 0:00:00
     ------------------------------------- 380.9/380.9 kB 23.2 MB/s eta 0:00:00
     ------------------------------------- 556.4/556.4 kB 36.4 MB/s eta 0:00:00
     ------------------------------------- 493.7/493.7 kB 15.6 MB/s eta 0:00:00
     ------------------------------------- 566.1/566.1 kB 34.7 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 28.3 MB/s eta 0:00:00
     ------------------------------------- 341.4/341.4 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 277.7/277.7 kB ? eta 0:00:00
     -------------------------------------- 158.6/158.6 kB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 64.7/64.7 kB ? eta 0:00:00
     ---------------------------------------- 8.9/8.9 MB 28.5 MB/s eta 0:00:00
     --------------------------------------- 41.3/41.3


[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Verificar instalación y versiones de librerías principales
missing = []
try:
    import torch
    print(f'Torch: {torch.__version__}')
except ImportError:
    print('Torch: NO INSTALADO')
    missing.append('torch')
try:
    import transformers
    print(f'Transformers: {transformers.__version__}')
except ImportError:
    print('Transformers: NO INSTALADO')
    missing.append('transformers')
try:
    import accelerate
    print(f'Accelerate: {accelerate.__version__}')
except ImportError:
    print('Accelerate: NO INSTALADO')
    missing.append('accelerate')
try:
    import peft
    print(f'PEFT: {peft.__version__}')
except ImportError:
    print('PEFT: NO INSTALADO')
    missing.append('peft')
try:
    import sentence_transformers
    print(f'Sentence-Transformers: {sentence_transformers.__version__}')
except ImportError:
    print('Sentence-Transformers: NO INSTALADO')
    missing.append('sentence-transformers')
if missing:
    print('❌ Faltan librerías:', ', '.join(missing))
else:
    print('✔ Todas las librerías principales están instaladas')

Torch: 2.2.2+cu121
Transformers: 4.57.3
Accelerate: 1.12.0
PEFT: 0.18.0
Sentence-Transformers: 5.2.0
✔ Todas las librerías principales están instaladas


In [15]:
import sys
print("Python ejecutable:", sys.executable)

Python ejecutable: c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\Scripts\python.exe


Reiniciar Kernels:
Ctrl + Shift + P
Jupyter: Restart Kernel

In [14]:
import transformers, accelerate, peft, torch

print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("PEFT:", peft.__version__)
print("Torch:", torch.__version__)

# assert transformers.__version__ == "4.38.2"
# assert accelerate.__version__ == "0.27.2"
# assert peft.__version__ == "0.9.0"

print("✔ Versiones correctas")


Transformers: 4.57.3
Accelerate: 1.12.0
PEFT: 0.18.0
Torch: 2.2.2+cu121
✔ Versiones correctas


In [17]:
%pip install datasets

     -------------------------------------- 511.6/511.6 kB 4.6 MB/s eta 0:00:00
     --------------------------------------- 11.3/11.3 MB 26.2 MB/s eta 0:00:00
     -------------------------------------- 119.7/119.7 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
     ---------------------------------------- 134.9/134.9 kB ? eta 0:00:00
     --------------------------------------- 28.1/28.1 MB 21.1 MB/s eta 0:00:00
     ------------------------------------- 455.1/455.1 kB 27.8 MB/s eta 0:00:00
     ---------------------------------------- 78.8/78.8 kB ? eta 0:00:00
     -------------------------------------- 113.4/113.4 kB 3.3 MB/s eta 0:00:00
     ------------------------------------- 348.5/348.5 kB 21.1 MB/s eta 0:00:00
     ------------------------------------- 509.2/509.2 kB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 41.6/41.6 kB ? eta 0:00:00
     ---------------------------------------- 43.8/43.8 kB 2.2 MB/


[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip uninstall -y numpy
%pip install numpy==1.26.4

Found existing installation: numpy 1.26.4Note: you may need to restart the kernel to use updated packages.

Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import random
import numpy as np
import torch

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

print("✔ Imports OK")


c:\Users\walte\Desktop\pln\proyecto_diplomado_pln\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✔ Imports OK


In [3]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ CUDA no disponible")

print("✔ Semilla fijada")


GPU: NVIDIA GeForce GTX 1070
✔ Semilla fijada


## 2. Cargar y preparar el conjunto de datos

Cargamos los archivos de AmericasNLP y el diccionario estructurado, y preparamos los pares (input, output) para el fine-tuning.

In [8]:
import pandas as pd

# Cargar AmericasNLP (train/dev/test)
train_path = 'AmericasNLP/guarani-train.tsv'
dev_path = 'AmericasNLP/guarani-dev.tsv'
test_path = 'AmericasNLP/guarani-test.tsv'

try:
    df_train = pd.read_csv(train_path, sep='\t')
    df_dev = pd.read_csv(dev_path, sep='\t')
    df_test = pd.read_csv(test_path, sep='\t')
    print('Train:', df_train.shape, '| Dev:', df_dev.shape, '| Test:', df_test.shape)
except Exception as e:
    print('Error cargando AmericasNLP:', e)

# Cargar diccionarios estructurados de la carpeta archivos
dict_ge_path = 'archivos/diccionario_guarani_espanol_estructurado.tsv'
dict_eg_path = 'archivos/diccionario_espanol_guarani_estructurado.tsv'

try:
    df_dict_ge = pd.read_csv(dict_ge_path, sep='\t')
    print('Diccionario guaraní-español:', df_dict_ge.shape)
except Exception as e:
    print('Error cargando diccionario guaraní-español:', e)

try:
    df_dict_eg = pd.read_csv(dict_eg_path, sep='\t')
    print('Diccionario español-guaraní:', df_dict_eg.shape)
except Exception as e:
    print('Error cargando diccionario español-guaraní:', e)

# Ejemplo de los primeros registros de cada diccionario
print('Ejemplo guaraní-español:')
print(df_dict_ge.head())
print('Ejemplo español-guaraní:')
print(df_dict_eg.head())

Train: (178, 4) | Dev: (79, 4) | Test: (364, 4)
Diccionario guaraní-español: (12073, 4)
Diccionario español-guaraní: (9425, 4)
Ejemplo guaraní-español:
  palabra_guarani tipo_etiqueta categoria_gramatical  \
0               a           NaN                  NaN   
1               a           NaN          f. v. ra s.   
2               ã           NaN                 m sa   
3               ã           NaN            adj. dem.   
4               ã           NaN                ta s.   

                                   traduccion_limpia  
0        Vocal que se pronuncia igual que en español  
1      Pref. a. de 1ª. Per. sin. para verbos propios  
2  Vocal que se pronuncia igual que la a pero con...  
3                                              Estos  
4                                              estas  
Ejemplo español-guaraní:
  palabra_espanol tipo_etiqueta categoria_gramatical  \
0               a           NaN                  NaN   
1               a           NaN             

In [9]:
# Preprocesar los diccionarios estructurados para tareas de traducción
if 'df_dict_eg' in locals():
    dict_data_eg = df_dict_eg[['palabra_espanol', 'traduccion_limpia']].copy()
    dict_data_eg = dict_data_eg.rename(columns={'palabra_espanol': 'input', 'traduccion_limpia': 'output'})
    print('Ejemplo español-guaraní estructurado:')
    print(dict_data_eg.head())
else:
    print('No se cargó el diccionario español-guaraní.')

if 'df_dict_ge' in locals():
    dict_data_ge = df_dict_ge[['palabra_guarani', 'traduccion_limpia']].copy()
    dict_data_ge = dict_data_ge.rename(columns={'palabra_guarani': 'input', 'traduccion_limpia': 'output'})
    print('Ejemplo guaraní-español estructurado:')
    print(dict_data_ge.head())
else:
    print('No se cargó el diccionario guaraní-español.')

Ejemplo español-guaraní estructurado:
         input                         output
0            a  f. Pu’ae avañe’ẽmeicha ipúva.
1            a         Pe, me,  ve, re, rehe.
2        abajo                  Yvýpe, guýpe.
3  abanderado,    da.  m y f. Poyvi rerahaha.
4  abandonado,              da.  adj. jeheja.
Ejemplo guaraní-español estructurado:
  input                                             output
0     a        Vocal que se pronuncia igual que en español
1     a      Pref. a. de 1ª. Per. sin. para verbos propios
2     ã  Vocal que se pronuncia igual que la a pero con...
3     ã                                              Estos
4     ã                                              estas


In [10]:
# Preprocesar AmericasNLP: crear pares (input, output) para fine-tuning

def build_input(row):
    return f"{row['Source']} [{row['Change']}]"

df_train = df_train.dropna(subset=['Source', 'Change', 'Target'])
df_dev = df_dev.dropna(subset=['Source', 'Change', 'Target'])

train_data = pd.DataFrame({
    'input': df_train.apply(build_input, axis=1),
    'output': df_train['Target']
})
dev_data = pd.DataFrame({
    'input': df_dev.apply(build_input, axis=1),
    'output': df_dev['Target']
})

# Concatenar los datos de los diccionarios estructurados para aumentar el set de entrenamiento
train_data = pd.concat([train_data, dict_data_eg, dict_data_ge], ignore_index=True)

print(train_data.head())

                                               input  \
0  Ha’e ombojerekuri umi kutuhakuéra poro’o [PERS...   
1  Ha’e ombojerekuri umi kutuhakuéra poro’o [TYPE...   
2  Ha’e ombojerekuri umi kutuhakuéra poro’o [ASPE...   
3    Mombe’ukuéra omboty kuri pende arete [TYPE:NEG]   
4  Mombe’ukuéra omboty kuri pende arete [ASPECT:I...   

                                            output  
0       Nde rembojerekuri umi kutuhakuéra tuicháva  
1    Ha’e ndombojereikuri umi kutuhakuéra tuicháva  
2  Ha’e ombojerehina kuri umi kutuhakuéra tuicháva  
3          Mombe’ukuéra ndombotyi kuri pende arete  
4        Mombe’ukuéra omboty kuri hína pende arete  


## 3. Tokenización de los datos

Utilizamos el tokenizer del modelo base para convertir los textos en tensores adecuados para el modelo.

In [11]:
# Cargar el tokenizer de Bloom-560m desde HuggingFace
model_name = 'bigscience/bloom-560m'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenización de los datos de entrenamiento y validación
def tokenize_function(example):
    return tokenizer(example['input'], truncation=True, padding='max_length', max_length=128)

# Convertir a HuggingFace Dataset
from datasets import Dataset, DatasetDict
train_hf = Dataset.from_pandas(train_data)
dev_hf = Dataset.from_pandas(dev_data)

datasets = DatasetDict({
    'train': train_hf,
    'validation': dev_hf
})

tokenized_datasets = datasets.map(tokenize_function, batched=True)
print(tokenized_datasets)

Map: 100%|██████████| 79/79 [00:00<00:00, 7180.78 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 21676
    })
    validation: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})


## 4. Configurar el modelo base para fine tuning

Cargamos el modelo preentrenado Llama-3.2-1B y lo preparamos para el ajuste fino.

In [12]:
# Cargar el modelo base para causal language modeling (Bloom-560m)
model = AutoModelForCausalLM.from_pretrained(model_name)

print('Modelo cargado:', model_name)

Modelo cargado: bigscience/bloom-560m


## 5. Definir hiperparámetros y configuración de entrenamiento

Establecemos los parámetros principales para el entrenamiento reproducible.

In [13]:
# training_args = TrainingArguments(
#     output_dir='./finetuned',
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=50,
#     seed=SEED,
#     report_to='none',
#     push_to_hub=False
# )

# print('Hiperparámetros definidos.')

training_args = TrainingArguments(
    output_dir='./finetuned',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    eval_strategy='no',
    save_strategy='epoch',
    logging_steps=25,
    logging_dir='./logs',
    weight_decay=0.01,
    seed=SEED,
    report_to='none',
    push_to_hub=False
)


## 6. Entrenar el modelo

Ejecutamos el proceso de entrenamiento usando los datos tokenizados y la configuración definida.

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    acc = np.mean([
        p.strip() == l.strip()
        for p, l in zip(decoded_preds, decoded_labels)
    ])

    return {"accuracy": acc}


In [15]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)


In [16]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["output"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [17]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

dev_dataset = dev_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dev_dataset.column_names
)


Map: 100%|██████████| 79/79 [00:00<00:00, 5265.46 examples/s]


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
)


C:\Users\walte\AppData\Local\Temp\ipykernel_6572\3903211903.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Step,Training Loss
25,13.451400
50,0.618500
75,0.785400
100,0.843700
125,0.604700
150,0.715000
175,0.818600
200,0.604900


## 7. Evaluar el modelo ajustado

Evaluamos el modelo fine-tuned en el conjunto de validación y mostramos métricas relevantes.

In [ ]:
# Evaluar el modelo en el conjunto de validación
eval_results = trainer.evaluate()
print('Resultados de evaluación:', eval_results)

## 8. Guardar el modelo fine-tuned

Guardamos el modelo entrenado y el tokenizer para su uso posterior o despliegue.

In [ ]:
# Guardar el modelo y el tokenizer
output_dir = './llama3-finetuned-final'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Modelo y tokenizer guardados en {output_dir}')

## 9. Inferencia: probar el modelo fine-tuned

Ejecuta el modelo ajustado sobre ejemplos nuevos o del set de test, como se requiere en la consigna del trabajo final.

In [ ]:
# Ejemplo de inferencia con el modelo fine-tuned
from transformers import pipeline

# Cargar modelo y tokenizer fine-tuned
dir_finetuned = './llama3-finetuned-final'
model = AutoModelForCausalLM.from_pretrained(dir_finetuned)
tokenizer = AutoTokenizer.from_pretrained(dir_finetuned)

# Ejemplo: usar un input del set de test o uno propio
test_example = "Ore ndorombyai kuri [TYPE:AFF]"
inputs = tokenizer(test_example, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=30)
pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Input:', test_example)
print('Predicción:', pred)